# Data Visualization in R
## by Diya Das

### The goal
Data visualization, like all visualization, is important because it tells a story. Take a moment to think about what stories you'd like to tell with your data. 

There's an important component of data visualization - deciding what kind of plot is appropriate to make a particular point - that we're not going to talk about today. In fact, today, we're going to make some visualizations that I consider to be pretty bad examples of data visualization, just to show you what is *technically* possible. However, as always, just because you *can* do something, it doesn't mean that you *should* do it. If you're interested in thinking about principles of good visualization, I encourage you to research that topic further. You could start with Edward Tufte.

I'm going to be giving examples of how to make plots in R, using both base R and the occasional `ggplot` command, as well as a 3D plotting library called `rgl`. (Why? because *why should you not gif your data?* Actually, it's because it's sometimes easier to look at relationships among high-dimensional data in 3D, vs. 2D.)

### The datasets
We are going to be using the data from the R package [`nycflights13`](https://cran.r-project.org/web/packages/nycflights13/nycflights13.pdf). There are five datasets corresponding to flights departing NYC in 2013. We will **load directly into R from the library**, but the repository also includes TSV files we created for the purposes of the Python demo and can also be used to load the data into our R session.

*** If you've never run Jupyter notebooks with R, please run `conda install -c r r-essentials`

In [ ]:
options(repos=structure(c(CRAN="http://cran.cnr.berkeley.edu/", 
BioCsoft="http://www.bioconductor.org/packages/release/bioc/")))
ipak <- function(pkg){
     new.pkg <- pkg[!(pkg %in% installed.packages()[, "Package"])]
     if (length(new.pkg))
         install.packages(new.pkg, dependencies = TRUE)
     sapply(pkg, require, character.only = TRUE)
 } #https://gist.github.com/stevenworthington/3178163

pkgs <- c("nycflights13", "ggplot2", "rgl", "NMF", "dplyr","tidyr")
ipak(pkgs)
options(jupyter.plot_mimetypes = 'image/png',repr.plot.width=6, repr.plot.height=3)

In [ ]:
#invisible(sapply(pkgs, library, character.only=TRUE )) 
    # if you knew you had installed the packages

## Basic scatterplots
Let's run through an example using the `flights` dataset. This dataset includes...well what does it include? You could read the documentation, but let's take a look first. 

What are the dimensions of the flights data frame?

In [ ]:
flights <- data.frame(flights) # dplyr has introduced a new data format that I am ignoring
flights <- flights[complete.cases(flights),] # remove NAs to reduce some issues
dim(flights)

Figure out the categories of data.

In [ ]:
summary(flights)

Let's just look at 1000 random flights in January.

In [ ]:
set.seed(5106)
flights <- flights[flights$month==1,]
flights <- flights[sample(1:nrow(flights),1000),]
dim(flights)

In [ ]:
plot(flights$dep_delay, flights$arr_delay)

## Changing aesthetic parameters

What if you prefer:
- filled circles instead of hollow: adjust `pch`
- different color points: adjust `col`
- different sized points: adjust `cex`
- different titles and axis labels: adjust `main` (title), `xlab`, `ylab`

In [ ]:
plot(flights$dep_delay, flights$arr_delay, cex=2, pch=19, 
     col="red", main= "Arrival vs. Departure Delay", 
     xlab= "Departure delay (minutes)", ylab="Arrival delay (minutes)")

## Coloring by one variable and adding a legend

In [ ]:
# Some tidying for plotting things
flights$origin <- factor(flights$origin) # convert to factor to color by origin in plot 
colpal <- scales::alpha(c("red","green","blue"),0.3) # how to make transparent colors

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
plot(flights$dep_delay, flights$arr_delay, cex=0.5, pch=19,
     col=colpal[flights$origin], main= "Arrival vs. Departure Delay")
legend("bottomright",levels(flights$origin),fill=colpal,
       cex=0.75, y.intersp=2)

## Histograms

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)
hist(flights$dep_time, main="Flights by time of departure", 
     xlab="Time", ylab = "# of flights in Jan 2013")

### Change number of divisions in a histogram

In [ ]:
hist(flights$dep_time, main="Flights by time of departure", 
     xlab="Time", ylab = "# of flights in Jan 2013",breaks=50)

### Plot proportions, not frequencies

In [ ]:
hist(flights$dep_time, main="Flights by time of departure", 
     xlab="Time", ylab = "Proportion of flights in Jan 2013",breaks=24, freq=FALSE)

## Plot organization and margins

What if you want to have two plots side by side, or change the margins?
In R, this is controlled by `par`.

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
par(mfrow=c(1,2)) # 1 row, 2 columns

plot(flights$dep_delay, flights$arr_delay, cex=0.5, pch=19,
     col=colpal[flights$origin], main= "Arrival vs. Departure Delay")
legend("bottomright",levels(flights$origin),fill=colpal,
       cex=0.75, y.intersp=2, xpd=FALSE)

hist(flights$dep_time, main="Flights by time of departure", 
     xlab="Time", ylab = "# of flights in 2013")


In [ ]:
par.defaults <- par() # save default plot parameters before we mess around
par.defaults

## Dot plots with jitter
## Turning off axes and labels

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
par(mfrow=c(2,1))
hist(flights$dep_time, main="Flights by time of departure", 
     xlab="Time", ylab = "# of flights in Jan 2013", col="purple")

par(mar=c(5.1,1,1,1)) # change margins for second plot
plot(flights$dep_time, jitter(rep(0, nrow(flights)), 0.3),
     main="Flights by time of departure", col=colpal[flights$origin],yaxt='n', ylab='',
    xlab = "Time")


# now reset to default margins and plot layout (1 x 1)
par(par.defaults)

## Boxplot numerical variable by categorical variable

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
boxplot(dep_time ~ origin, data=flights, col=colpal, ylab='Departure time')

## Plotting multiple series
### Base R

In [ ]:
delay_time <- flights %>% group_by(origin,time_hour) %>% 
    summarise(avg_dep_delay=mean(dep_delay), avg_arr_delay=mean(arr_delay))
print(head(delay_time))

In [ ]:
with(subset(delay_time, origin=='EWR'), plot(time_hour, avg_dep_delay, type="l",col=colpal[1],
                                            main="Average departure delay by airport",
                                            ylab = "Departure delay", xlab="Time"))
with(subset(delay_time, origin=='JFK'), lines(time_hour, avg_dep_delay,col=colpal[2]))
with(subset(delay_time, origin=='LGA'), lines(time_hour, avg_dep_delay,col=colpal[3]))

legend("top",levels(flights$origin),fill=colpal,
       cex=0.75, y.intersp=2, x.intersp=2, horiz=TRUE)

### `ggplot`

In [ ]:
ggplot(delay_time,aes(x=time_hour,y=avg_dep_delay,group=origin)) + geom_line(aes(colour = origin)) +
ggtitle('Average departure delay vs. Time') +
xlab('Time') + ylab('Arrival departure delay (minutes)')

## Bubble plots and diving into `ggplot`
Bubble plots are useful for plotting three variables at once, where the third variable is some numerical quantity.

Let's plot flights by origin and carrier. First, we need to construct a long-formatted data frame for input to `ggplot` (see Data Tidying and Manipulation notebook).

In [ ]:
origin_carrier <- reshape2::melt(
    prop.table(table(Origin=factor(flights$origin), 
                     Carrier=factor(flights$carrier)), 1), 
               value.name="Proportion")

head(origin_carrier)
rowSums(prop.table(table(Origin=factor(flights$origin), 
                     Carrier=factor(flights$carrier)), 1))
# Each carrier as a fraction of flights out of that airport

## `ggplot`: A conceptual introduction

`ggplot` is based on the **grammar of graphics** - that is, a component-based approach to building graphics:

- data
- coordinate system
- geoms: visual representation of data
    - aesthetics: size, color, x, y locations

The template is as follows, from the very helpful cheatsheet from RStudio:

`ggplot(data = <DATA>) + 
  <GEOM_FUNCTION>(mapping=aes(<MAPPINGS>),stat= <STAT>,position=<POSITION>) +
  <COORDINATE_FUNCTION> + <FACET_FUNCTION> + <SCALE_FUNCTION> + <THEME_FUNCTION>`

Create the base plot, describing what data are to be plotted. Note the size statement, with the `ifelse`: this is to alter the default behavior of `ggplot` which plots zeros as a tiny point, to distinguish them from `NA`s.

In [ ]:
or_car.plt <- ggplot(origin_carrier, 
                     aes(Carrier, Origin, size=ifelse(Proportion==0, NA, Proportion)))

Now, for plotting:

- `geom_point` is a `GEOM_FUNCTION` that tells `ggplot` we want to plot points.
- `scale_size_area` is a `SCALE_FUNCTION`

In [ ]:
pdf("bubble.pdf", width=6, height=4)
plt2 <- or_car.plt + geom_point(color = "blue") + 
   scale_size_area(max_size=20, name='Proportion', breaks=c(0.1, 0.25, 0.5))
plt2
dev.off()

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)
theme1 <- theme(plot.background=element_blank(), panel.grid.minor=element_blank(), 
            panel.border=element_blank(), panel.background=element_blank(), 
            axis.line=element_blank(),axis.ticks=element_blank(), 
            legend.title=element_blank(), legend.background=element_blank(), 
            axis.text.x=element_blank(), axis.text.y=element_blank(),
            legend.key= element_rect(fill="white"))
plt2 + theme1

In [ ]:
theme2 <- theme(plot.background=element_blank(), 
            panel.background=element_blank(), 
            legend.background=element_blank(), 
            legend.key= element_rect(fill="white"))
plt2 + theme2

## Stacked bar plots

In [ ]:
ggplot(origin_carrier, aes(x = Carrier, y = Proportion*100, fill = factor(Origin))) +
  geom_bar(stat = "identity") + 
  scale_fill_manual(values=colpal, name="Origin") + theme_bw() + 
  ggtitle('Proportion of flights from each airport by carrier') +
  theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
      panel.border=element_blank(),
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1)) +ylab("Percent")

## More with `ggplot`: Going back over some of the graphics we've already made in base R

### Basic scatterplots

In [ ]:
delay.plt <- ggplot(data=flights, aes(dep_delay, arr_delay,colour=origin)) 
delay.plt + geom_point(size=0.5) + 
ggtitle('Arrival delay vs. Departure delay') + theme(plot.title = element_text(hjust = 0.5)) +
theme_bw() +
xlab('Departure delay (minutes)') + ylab('Arrival delay (minutes)') +
scale_colour_manual(values=colpal)+
geom_smooth()

### Histograms

In [ ]:
ggplot(data=flights, aes(dep_time)) + geom_histogram(binwidth=60) + 
  ggtitle("Flights by time of departure")+ xlab("Time") + ylab ("# of flights in Jan 2013")

### Dot plots

In [ ]:
ggplot(data=flights, aes(dep_time, rep(0, nrow(flights))))+ geom_jitter(aes(colour = origin)) +
  ggtitle('Flights by time of departure') + xlab('Time') +
  scale_y_continuous(breaks=NULL) + ylab('')
suppressWarnings(par(par.defaults))

### Boxplots

In [ ]:
ggplot(data=flights, aes(origin,dep_time)) + geom_boxplot(fill=colpal) + ylab('Departure time')

## Faceting with `ggplot2`

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)
delay.plt + geom_point() + geom_smooth() + facet_grid(. ~ origin) +
ylab('Arrival delay') + xlab('Departure delay')

## Heatmaps

In [ ]:
airtime <- left_join(flights, airports, by = c("dest" = "faa")) %>% 
    select(origin, dest=name, air_time) %>% 
    group_by(origin, dest) %>% 
    summarize(avg_air_time = mean(air_time))
avg_air <- data.frame(drop_na(spread(airtime, origin, avg_air_time)))
rownames(avg_air) <- avg_air$dest
head(avg_air)

In [ ]:
options(repr.plot.width=6, repr.plot.height=5)
terminals <- c("3","6","4") # Number of current terminals, as per Wikipedia
aheatmap(avg_air[,2:ncol(avg_air)], annCol=data.frame(num_terminals=terminals), cexCol=0.8)

## 3D plotting

In [ ]:
plot3d(flights$distance, flights$air_time, flights$arr_delay)
par3d(windowRect = c(20, 30, 800, 800))
print(getwd())
movie3d(spin3d(), 5, movie = "movie", frames = "movie", dir = getwd(), 
                    convert = TRUE, clean = TRUE, verbose=TRUE,
                    top = TRUE, type = "gif") 
rgl.close()

## Pairs plots: 2D representations of high-dimensional data

In [ ]:
pairs(flights[,c("distance","air_time", "arr_delay")], col=colpal[flights$origin])
legend("bottomright",levels(flights$origin),fill=colpal,cex=0.5, xpd=TRUE)